Merging divisions per season and saving in csv fil

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
def load_merge_data():
    df = pd.DataFrame()
    bool=False
    for subdir, dirs, files in os.walk('.'):
        #print(subdir)
        for file in files:
            #print(file)
            ext = os.path.splitext(file)[-1].lower()
            if ext == ".csv":
                datapath=os.path.join(subdir, file)
                #print(datapath)
                try:
                    data=pd.read_csv(datapath, header=0)
                    print(data.head())
                    #print(data)
                    df=pd.concat([df, data], axis=0, ignore_index=True)
                    bool=True
                    #print(d)
                    #print(datapath)
                except Exception:
                    print("Cannot read files: ",datapath)
        if bool:
            df.to_csv(subdir + '.csv')
        df = pd.DataFrame()
        
        return df
        

In [3]:
#data=pd.read_csv('.\csv files\Season 1993_1994\D1.csv', header=0, index_col=0)
#data.head()
df = load_merge_data()

   Unnamed: 0  id  team_api_id  team_fifa_api_id     team_long_name  \
0           0   1         9987             673.0           KRC Genk   
1           1   2         9993             675.0       Beerschot AC   
2           2   3        10000           15005.0   SV Zulte-Waregem   
3           3   4         9994            2007.0   Sporting Lokeren   
4           4   5         9984            1750.0  KSV Cercle Brugge   

  team_short_name  
0             GEN  
1             BAC  
2             ZUL  
3             LOK  
4             CEB  
   Unnamed: 0  team_api_id  team_rank team_long_name  team_offense_rating  \
0           0         8634          1      Barcelona                  3.3   
1           1         8633          2    Real Madrid                  3.4   
2           2         9823          3  Bayern Munich                  3.2   
3           3         9847          4            PSG                  3.2   
4           4         8456          5      Man. City                

#### Scrape Data For Features 


In [4]:
import sqlite3

database = sqlite3.connect('database.sqlite')

#look up table names
all_tables = pd.read_sql("SELECT * FROM sqlite_master;", database)
#print(all_tables)

players = all_tables.get(4)
players = pd.read_sql("SELECT * FROM Player;", database)
matches = pd.read_sql("SELECT * FROM Match;", database)
leagues = pd.read_sql("SELECT * FROM League;", database)
countries = pd.read_sql("SELECT * FROM Country;", database)
teams = pd.read_sql("SELECT * FROM Team;", database)
team_attributes = pd.read_sql("SELECT * FROM Team_Attributes;", database)
player_attributes = pd.read_sql("SELECT * FROM Player_Attributes;",database)

table_list = [players, matches, leagues, countries, teams, team_attributes, player_attributes]
total_list = pd.DataFrame()
#for table in table_list:
#   print '\n',table.info(),table.head()
    
total_list = pd.concat(table_list, axis=1, keys=['players','matches','leagues','countries','teams','team_attributes','player_attributes'])

print teams.head()

   id  team_api_id  team_fifa_api_id     team_long_name team_short_name
0   1         9987             673.0           KRC Genk             GEN
1   2         9993             675.0       Beerschot AC             BAC
2   3        10000           15005.0   SV Zulte-Waregem             ZUL
3   4         9994            2007.0   Sporting Lokeren             LOK
4   5         9984            1750.0  KSV Cercle Brugge             CEB


In [5]:
# total_list.to_csv('mergedlist.csv', encoding='utf-8')

team_barcelona = teams.loc[teams['team_long_name'] == 'FC Barcelona']

print team_barcelona.loc[:'team_api_id']
print team_barcelona.iloc[0]['team_api_id']

teams.to_csv('teams.csv', encoding='utf-8')


        id  team_api_id  team_fifa_api_id team_long_name team_short_name
258  43042         8634             241.0   FC Barcelona             BAR
8634


### Team Rating Scrapping
- Team fifa ratings
- NLP team sentiment

In [6]:
from bs4 import BeautifulSoup
import requests
import urllib, json

print teams.head()

   id  team_api_id  team_fifa_api_id     team_long_name team_short_name
0   1         9987             673.0           KRC Genk             GEN
1   2         9993             675.0       Beerschot AC             BAC
2   3        10000           15005.0   SV Zulte-Waregem             ZUL
3   4         9994            2007.0   Sporting Lokeren             LOK
4   5         9984            1750.0  KSV Cercle Brugge             CEB


In [8]:
def dataframeColumns():
    return ["team_api_id","team_rank","team_long_name","team_offense_rating","team_defense_rating","team_spi_rating"]

def getTeamId(team_name, teams_csv):
    returned_id = teams_csv.loc[teams_csv['team_short_name'].str.contains(team_name)]
    if (returned_id.empty):
        returned_id = teams_csv.loc[teams_csv['team_long_name'].str.contains(team_name)]
    if (returned_id.empty):
        return 0
    else:
        return returned_id.iloc[0]['team_api_id']
    

In [9]:
team_name = teams.loc[teams['team_api_id'] == 9987]

name = team_name['team_long_name']

# tweet_criteria = GetOldTweets.got.manager.TweetCriteria().setQuerySearch(name.get(0)).setSince("2010-01-01").setMaxTweets(100)

dataframe = pd.DataFrame(columns=dataframeColumns())
webpage = "Global Club Soccer Rankings _ FiveThirtyEight.htm"
#page = urllib.urlopen('https://projects.fivethirtyeight.com/global-club-soccer-rankings/')

with open(webpage) as fp:
    soup = BeautifulSoup(fp, 'lxml')
    
team_names =  soup.find_all('div', attrs={'class':'name'})
    
team_offense_ratings = soup.find_all('td', attrs={'class':'num rating offense drop-6'})

#for b in team_offense_ratings:
    #print b.text
    
team_defense_ratings = soup.find_all('td', attrs={'class':"num rating defense drop-6"})
team_spi_ratings = soup.find_all('td', attrs={'class':"num"})

team_spi_parsed = []
print 'SPI Ratings\n'
for c in range(3, len(team_spi_ratings), 4):
    team_spi_parsed.append(float(team_spi_ratings[c].contents[0]))


team_ranking = soup.find_all('td', attrs={'class':'num ranking'})

for i in range(len(team_names)):
    team_api_id = getTeamId(team_names[i].text, teams)
    dataframe.loc[len(dataframe)] = [team_api_id, team_ranking[i].text, team_names[i].text, team_offense_ratings[i].text, team_defense_ratings[i].text, team_spi_parsed[i]]


team_manchester = teams.loc[teams['team_long_name'] == 'Manchester City']

api_id = teams.iloc[team_manchester.index]['team_api_id']
api = api_id.get_values()

man_loc = dataframe.loc[dataframe['team_long_name'] == 'Man. City']

dataframe.loc[man_loc.index,'team_api_id'] = api[0]

print dataframe.loc[dataframe['team_long_name'] == 'Man. City']['team_api_id']


team_united = teams.loc[teams['team_long_name'] == 'Manchester United']
api_id = teams.iloc[team_united.index]['team_api_id']
api = api_id.get_values()
united_loc = dataframe.loc[dataframe['team_long_name'] == 'Man. United']
dataframe.loc[united_loc.index,'team_api_id'] = api[0]
print dataframe.loc[dataframe['team_long_name'] == 'Man. United']['team_api_id']

dataframe.to_csv('team_rankings.csv', encoding='utf-8')



SPI Ratings

4    8456
Name: team_api_id, dtype: object
12    10260
Name: team_api_id, dtype: object


### Adding New Features To Large Dataset

In [10]:
final_csv = pd.read_csv('Data/FINAL_ALL_FEATURES_NO_NAN_final.csv')

new_cols = ['home_team_rating','away_team_rating','home_team_offensive_rating','away_team_offensive_rating','home_team_defensive_rating','away_team_defensive_rating']
for col in new_cols:
    final_csv[col] = 0
final_csv = final_csv.drop('Unnamed: 0',axis=1)
i = 0;
first_col_id = final_csv.loc[i,'home_team_api_id']
first_col_matching = dataframe.loc[dataframe['team_api_id'] == first_col_id]

print first_col_matching.index
print first_col_matching['team_spi_rating'].values[0]
print first_col_id
print final_csv.index




Int64Index([85], dtype='int64')
59.3
8635
RangeIndex(start=0, stop=17369, step=1)


In [11]:
rating_mean = dataframe['team_spi_rating'].mean()
#rating_mean = rating_mean.mean()
print rating_mean

offense_mean = dataframe['team_offense_rating']
offense_mean = pd.to_numeric(offense_mean)
print offense_mean.mean()

defense_mean = dataframe['team_defense_rating']
defense_mean = pd.to_numeric(defense_mean)
print defense_mean.mean()




46.2876379691
1.35651214128
1.3174392936


In [36]:

for index in final_csv.index:
    row_home_id = final_csv.iloc[index, final_csv.columns.get_loc('home_team_api_id')]
    home_col_matching = dataframe.loc[dataframe['team_api_id'] == row_home_id]
    if (home_col_matching.empty):
        final_csv.iloc[index, final_csv.columns.get_loc('home_team_rating')] = rating_mean
        final_csv.iloc[index, final_csv.columns.get_loc('home_team_offensive_rating')] = offense_mean.mean()
        final_csv.iloc[index, final_csv.columns.get_loc('home_team_defensive_rating')] = defense_mean.mean()
    else:
        final_csv.iloc[index, final_csv.columns.get_loc('home_team_rating')] = home_col_matching['team_spi_rating'].values[0]
        final_csv.iloc[index, final_csv.columns.get_loc('home_team_offensive_rating')] = home_col_matching['team_offense_rating'].values[0]
        final_csv.iloc[index, final_csv.columns.get_loc('home_team_defensive_rating')] = home_col_matching['team_defense_rating'].values[0]
    row_away_id = final_csv.iloc[index, final_csv.columns.get_loc('away_team_api_id')]
    away_col_matching = dataframe.loc[dataframe['team_api_id'] == row_away_id]
    if (away_col_matching.empty):
        final_csv.iloc[index, final_csv.columns.get_loc('away_team_rating')] = rating_mean
        final_csv.iloc[index, final_csv.columns.get_loc('away_team_offensive_rating')] = offense_mean.mean()
        final_csv.iloc[index, final_csv.columns.get_loc('away_team_defensive_rating')] = defense_mean.mean()
    else:
        final_csv.iloc[index, final_csv.columns.get_loc('away_team_rating')] = away_col_matching['team_spi_rating'].values[0]
        final_csv.iloc[index, final_csv.columns.get_loc('away_team_offensive_rating')] = away_col_matching['team_offense_rating'].values[0]
        final_csv.iloc[index, final_csv.columns.get_loc('away_team_defensive_rating')] = away_col_matching['team_defense_rating'].values[0]
    if ((index%1000) == 0):
        print 'Working... ',index

final_csv.head()


Working...  0
Working...  1000
Working...  2000
Working...  3000
Working...  4000
Working...  5000
Working...  6000
Working...  7000
Working...  8000
Working...  9000
Working...  10000
Working...  11000
Working...  12000
Working...  13000
Working...  14000
Working...  15000
Working...  16000
Working...  17000


,id,match_api_id,home_team_api_id,away_team_api_id,season,date,result_label,away_Defender_rating,home_Defender_rating,away_Attacker_rating,...,HOME_DRAW_RATE_THIS_SEASON,AWAY_DRAW_RATE,AWAY_DRAW_RATE_THIS_SEASON,AWAY_DRAW_RATE_AT_THIS_GROUND,home_team_rating,away_team_rating,home_team_offensive_rating,away_team_offensive_rating,home_team_defensive_rating,away_team_defensive_rating
0,540,838599,8635,10001,2010/2011,2010-10-24,HOME_WIN,64.500000,72.666667,67.666667,...,0.200000,0.181818,0.25,0.000000,59.300000,46.287638,1.7,1.35651,1.1,1.31744
1,549,838622,9991,8203,2010/2011,2010-10-30,HOME_WIN,64.600000,65.666667,66.500000,...,0.000000,0.066667,0.20,0.000000,46.287638,46.287638,1.35651,1.35651,1.31744,1.31744
2,552,838625,10000,9994,2010/2011,2010-10-30,DRAW,64.000000,64.333333,67.000000,...,0.200000,0.153846,0.50,0.145833,46.287638,69.700000,1.35651,1.9,1.31744,0.9
3,563,838646,9984,9991,2010/2011,2010-11-07,AWAY_WIN,63.000000,64.750000,68.500000,...,0.166667,0.200000,0.00,1.000000,46.287638,46.287638,1.35651,1.35651,1.31744,1.31744
4,569,838658,9986,10001,2010/2011,2010-11-13,AWAY_WIN,64.666667,61.000000,67.333333,...,0.000000,0.166667,0.20,1.000000,46.287638,46.287638,1.35651,1.35651,1.31744,1.31744


In [12]:
final_csv = pd.read_csv('../Preprocessing/Data_Structures/FINAL_BASE_DATASET_TEAM_RATINGS_WITH_NO_ZEROS_ADDED_NO_BETS_AGE_BMI_ZEROS_NOT_REMOVED.csv', encoding='utf-8')

In [13]:
final_csv = final_csv.drop('Unnamed: 0', axis=1)

final_csv.head()


,away_Defender_rating,home_Defender_rating,away_Attacker_rating,home_Attacker_rating,away_Goalkeeper_rating,home_Goalkeeper_rating,away_Midfielder_rating,home_Midfielder_rating,HEAD_2_HEAD_HOME_TEAM_WINS,HEAD_2_HEAD_HOME_TEAM_LOSS,...,AWAY_DRAW_RATE,AWAY_DRAW_RATE_THIS_SEASON,AWAY_DRAW_RATE_AT_THIS_GROUND,Label,home_team_rating,away_team_rating,home_team_offensive_rating,away_team_offensive_rating,home_team_defensive_rating,away_team_defensive_rating
0,64.500000,72.666667,67.666667,74.0,63.0,72.0,64.333333,73.833333,1.000,0.0,...,0.181818,0.25,0.000000,2,59.300000,46.287638,1.700000,1.356512,1.100000,1.317439
1,64.600000,65.666667,66.500000,69.0,69.0,71.0,66.000000,69.600000,1.000,0.0,...,0.066667,0.20,0.000000,2,46.287638,46.287638,1.356512,1.356512,1.317439,1.317439
2,64.000000,64.333333,67.000000,66.0,63.0,69.0,65.400000,65.600000,0.000,0.0,...,0.153846,0.50,0.145833,1,46.287638,69.700000,1.356512,1.900000,1.317439,0.900000
3,63.000000,64.750000,68.500000,65.0,71.0,65.0,69.666667,65.500000,0.000,1.0,...,0.200000,0.00,1.000000,0,46.287638,46.287638,1.356512,1.356512,1.317439,1.317439
4,64.666667,61.000000,67.333333,0.0,68.0,58.0,65.250000,61.000000,0.125,0.5,...,0.166667,0.20,1.000000,0,46.287638,46.287638,1.356512,1.356512,1.317439,1.317439


### Random Forest


In [19]:
from sklearn.model_selection import train_test_split



Youtput = final_csv.loc[:,'Label']
xset = final_csv.drop('Label', axis=1)
Xinput = xset.loc[:,'away_Defender_rating':]
X_train, X_test, y_train, y_test = train_test_split(Xinput, Youtput)

X_train.head()
X_test.head()

,away_Defender_rating,home_Defender_rating,away_Attacker_rating,home_Attacker_rating,away_Goalkeeper_rating,home_Goalkeeper_rating,away_Midfielder_rating,home_Midfielder_rating,HEAD_2_HEAD_HOME_TEAM_WINS,HEAD_2_HEAD_HOME_TEAM_LOSS,...,HOME_DRAW_RATE_THIS_SEASON,AWAY_DRAW_RATE,AWAY_DRAW_RATE_THIS_SEASON,AWAY_DRAW_RATE_AT_THIS_GROUND,home_team_rating,away_team_rating,home_team_offensive_rating,away_team_offensive_rating,home_team_defensive_rating,away_team_defensive_rating
6774,76.4,75.000000,78.5,75.0,71.0,71.0,76.666667,71.500000,0.000000,0.333333,...,0.312500,0.258065,0.187500,0.000000,70.200000,71.900000,1.800000,2.100000,0.700000,0.900000
12188,71.0,72.666667,71.0,76.0,73.0,78.0,70.400000,75.600000,0.400000,0.200000,...,0.083333,0.209877,0.083333,0.000000,73.000000,46.287638,2.600000,1.356512,1.200000,1.317439
13360,65.0,64.750000,68.0,67.5,69.0,67.0,67.000000,65.250000,0.330000,0.330000,...,0.428571,0.000000,0.000000,0.072789,46.287638,44.300000,1.356512,1.400000,1.317439,1.500000
11731,69.0,63.400000,69.5,65.0,68.0,69.0,71.000000,66.333333,0.000000,1.000000,...,0.200000,0.326087,0.285714,1.000000,46.287638,55.400000,1.356512,1.700000,1.317439,1.200000
15930,73.5,75.500000,77.5,71.0,75.0,75.0,77.000000,72.400000,0.222222,0.555556,...,0.214286,0.272727,0.461538,0.750000,64.400000,76.500000,1.700000,2.400000,0.900000,0.900000


In [15]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=1000, oob_score=True, random_state=123456)


rf_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=True,
           random_state=123456, verbose=0, warm_start=False)

In [39]:
y_test.head()


array([ nan])

In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn import metrics

X_train.head()
predicted = rf_model.predict(X_test)
predict_frame = pd.DataFrame(predicted, columns=['Label'])
test_frame = pd.DataFrame(y_test, columns=['Label'])
testing_error = mean_squared_error(y_test, predicted)
predict_frame.head()
#test_frame.head()


#accuracy_score = accuracy_score(y_test, predicted)

,Label
0,1.146
1,0.836
2,1.489
3,0.930
4,0.981


In [18]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, predicted, pos_label=1)
measured = metrics.auc(fpr,tpr)
measured
accuracy_score(y_test,predicted)

ValueError: Can't handle mix of multiclass and continuous

In [91]:
import xgboost as xgb

xgb_new = xgb.XGBClassifier(learning_rate=0.1, n_estimators=85, max_depth=10)
xgb_new.fit(X_train, y_train)

predict = xgb_new.predict_proba(X_test)
predict
y_test


2457     2
2365     1
8615     2
2367     2
11037    0
3554     0
4042     2
5536     2
5649     1
4263     1
2812     2
5799     1
14223    2
8503     2
5843     0
4060     1
12905    2
15885    2
304      1
6286     0
2267     2
14325    2
4308     1
15970    0
9325     1
13687    2
3301     1
15324    2
7577     0
17222    1
        ..
9029     1
9875     2
1393     2
10996    1
16409    2
49       2
5969     2
14868    1
17105    2
15016    1
1786     0
11740    2
941      2
1130     1
6835     1
6585     1
13722    2
13285    0
13282    2
4649     2
17005    1
8527     0
8747     1
10882    2
10090    0
9896     1
3690     2
12182    1
11428    0
8622     1
Name: Label, Length: 4340, dtype: int64

In [92]:
predict = predict.T
predict = predict[1]
print predict
fpr, tpr, thresholds = metrics.roc_curve(y_test, predict, pos_label=1)
measured = metrics.auc(fpr,tpr)
measured

[ 0.10566352  0.14514807  0.16376969 ...,  0.20681329  0.24490601
  0.2244224 ]


0.60174252360386149

In [79]:
prediction = predict[0]
prediction
y_test
predict
prediction = predict.T
prediction = prediction[0]
prediction.size

4340

In [22]:
from sklearn.model_selection import GridSearchCV

xgb = xgb.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth = 5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
param_test = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2),
    'learning_rate':[0.1,0.01,0.06],
    'n_estimators':[10,51]
}

gridCV = GridSearchCV(estimator=xgb, param_grid=param_test, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gridCV.fit(X_train,y_train)

predictCV = gridCV.predict_proba(X_test)
predictCV

NameError: name 'xgb' is not defined